#### Comparisons for per residue data

The contacts dataset here already has msa numbering

In [1]:
import re
import numpy as np
import pandas as pd
import statistics

In [2]:
# code taken from Dariia
def parse_fasta(file_path):
    """Takes in a msa alignment produced by modeller in fasta formating and
    outputs a sequence as string with '-' for the gaps introduced by msa"""
    sequences = {}
    current_name = None
    current_sequence = []

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):
                if current_name is not None:
                    sequences[current_name] = "".join(current_sequence)
                    current_sequence = []

                current_name = line[1:]
            else:
                current_sequence.append(line)

        # Add the last sequence to the dictionary
        if current_name is not None:
            sequences[current_name] = "".join(current_sequence)

    return sequences

### Part 1: Setup steps

In [3]:
CONTACTS_PATH = r"../../contact_analysis/multi_structure_test/msa_outputs/" 
CRYSTAL_STRUCTURES_PATH = r"../../file_list.txt"
MSA_DATA_PATH = r"../../contact_analysis/multi_structure_test/bettaLac.ali"
LARGEST_MSA_RES = 295 # largest msa residue number in any sequence. 

In [4]:
# all xray structures to analyse
with open(CRYSTAL_STRUCTURES_PATH, "r", encoding="utf-8") as file:
    crystal_structures = file.read().splitlines()
len(crystal_structures), crystal_structures[0:3]

(69, ['1BSG_SAL-1', '1BUE_NmcA', '1DY6_SME-1'])

In [5]:
# msa sequences for all structures
msa_sequences = parse_fasta(MSA_DATA_PATH)
print(msa_sequences)

{'1BSG_SAL-1': '----S-------DAERRLAGLERASGARLGVYAYDTGSGRTVAYRADELFPMCSVFKTLSSAAVLRDLDRNGEFLSRRILYTQDDVEQADGAGPETGKPQNLANAQ-LTVEELCEVSITASDNCAANLMLRELGGPAA----VTRFVRSLGDRVTRLDRWEPELNSAEPGRVTDTTSPRAITRTYGRLVLGD-ALNPRDRRLLTSWLLANTTSGDRFRAGLPDDWTLGDKTGA-GRYGTNNDAGVTWPPGRAPIVLTVLTAKTEQDAARDD--GLVADAARVLAETL-------G*', '1BUE_NmcA': '----NT----KGIDE--IKNLETDFNGRIGVYALDTGSGKSFSYRANERFPLCSSFKGFLAAAVLKGSQDN-RLNLNQIVNYN-----TRSLEFHSPITTKYKDNG-MSLGDMAAAALQYSDNGATNIILERYIGGPEG---MTKFMRSIGDEDFRLDRWELDLNTAIPGDERDTSTPAAVAKSLKTLALGN-ILSEHEKETYQTWLKGNTTGAARIRASVPSDWVVGDKTGSCGAYGTANDYAVVW-PKNRAPLIISVYTTKNEKEAKHEDKVIAEASRIAIDNLK--------*', '1DY6_SME-1': '----NKSD--AAAKQ--IKKLEEDFDGRIGVFAIDTGSGNTFGYRSDERFPLCSSFKGFLAAAVLERVQQK-KLDINQKVKYE-----SRDLEYHSPITTKYKGSG-MTLGDMASAALQYSDNGATNIIMERFLGGPEG---MTKFMRSIGDNEFRLDRWELELNTAIPGDKRDTSTPKAVANSLNKLALGN-VLNAKVKAIYQNWLKGNTTGDARIRASVPADWVVGDKTGSCGAYGTANDYAVIW-PKNRAPLIVSIYTTRKSKDDKHSDKTIAEASRIAIQAID--------*', '1E25_PER-1': '--SPLLK--EQ------IESIVIGKKATVGVAVWGPDDLEPLL

In [6]:
len(msa_sequences['1BSG_SAL-1'])

296

In [7]:
msa_sequences['1BSG_SAL-1']

'----S-------DAERRLAGLERASGARLGVYAYDTGSGRTVAYRADELFPMCSVFKTLSSAAVLRDLDRNGEFLSRRILYTQDDVEQADGAGPETGKPQNLANAQ-LTVEELCEVSITASDNCAANLMLRELGGPAA----VTRFVRSLGDRVTRLDRWEPELNSAEPGRVTDTTSPRAITRTYGRLVLGD-ALNPRDRRLLTSWLLANTTSGDRFRAGLPDDWTLGDKTGA-GRYGTNNDAGVTWPPGRAPIVLTVLTAKTEQDAARDD--GLVADAARVLAETL-------G*'

In [8]:
# protein contacts found and formatted
all_residue_contacts = {}
for protein in crystal_structures:

    contacts_file_path = CONTACTS_PATH + protein + "_msa.csv"
    with open(contacts_file_path, "r", encoding="utf-8") as file:
        contacts = file.read().splitlines()[1:] # skip topline as header. 

    residue_contacts = []
    for contact in contacts:
        res1_numb = int(contact.split(",")[4])
        res2_numb = int(contact.split(",")[5])
        residue_contacts.append( (res1_numb, res2_numb) )

    # list of tuples to specificy contacts
    all_residue_contacts[protein] = residue_contacts

print(all_residue_contacts)

{'1BSG_SAL-1': [(5, 16), (13, 17), (14, 18), (14, 42), (14, 282), (14, 286), (15, 19), (15, 43), (15, 44), (16, 20), (17, 21), (17, 282), (17, 285), (17, 286), (18, 22), (18, 29), (18, 31), (18, 42), (18, 44), (18, 279), (18, 282), (19, 23), (19, 44), (20, 24), (21, 25), (21, 275), (21, 278), (21, 279), (21, 282), (22, 26), (22, 27), (22, 28), (22, 29), (22, 44), (22, 45), (25, 262), (25, 270), (25, 275), (27, 260), (27, 261), (27, 262), (28, 45), (28, 48), (28, 49), (28, 50), (28, 259), (28, 260), (28, 261), (29, 31), (29, 44), (29, 45), (29, 50), (29, 258), (29, 259), (29, 260), (29, 275), (30, 44), (30, 45), (30, 50), (30, 178), (30, 257), (30, 258), (30, 259), (31, 42), (31, 43), (31, 44), (31, 46), (31, 178), (31, 257), (31, 258), (31, 283), (32, 41), (32, 42), (32, 43), (32, 46), (32, 178), (32, 179), (32, 182), (32, 255), (32, 256), (32, 257), (33, 40), (33, 41), (33, 42), (33, 255), (33, 256), (33, 283), (33, 287), (34, 39), (34, 40), (34, 41), (34, 253), (34, 254), (34, 255), 

### Step 2: Calculate the coeffcient of variation for each residue 

In [9]:
all_interaction_counts = {}
for msa_res_numb in range(1, LARGEST_MSA_RES+1):
    interaction_counts = []
    for protein, contacts in all_residue_contacts.items():

        # check if this residue exists in this protein. 
        msa_residue = msa_sequences[protein][msa_res_numb-1] # 0-indexed
        if msa_residue != "-": # i.e. does exist. 

            # unravelled, so long list that can just be counted
            unravelled_contacts = [item for t in contacts for item in t]
            numb_contacts_formed = unravelled_contacts.count(msa_res_numb)
            interaction_counts.append(numb_contacts_formed)
        
        else: 
            interaction_counts.append("res not in seq")

    all_interaction_counts[msa_res_numb] = interaction_counts
print(all_interaction_counts[5])

[1, 2, 1, 3, 0, 'res not in seq', 4, 'res not in seq', 3, 4, 0, 2, 'res not in seq', 'res not in seq', 2, 4, 2, 'res not in seq', 'res not in seq', 2, 'res not in seq', 4, 'res not in seq', 1, 1, 1, 1, 'res not in seq', 1, 'res not in seq', 1, 'res not in seq', 'res not in seq', 1, 'res not in seq', 5, 'res not in seq', 'res not in seq', 2, 1, 3, 0, 2, 'res not in seq', 2, 4, 1, 5, 3, 'res not in seq', 'res not in seq', 3, 1, 3, 1, 1, 2, 3, 3, 1, 0, 0, 'res not in seq', 3, 3, 0, 1, 4, 3]


In [34]:
coeff_variantions = {}
for msa_res_numb, interaction_counts in all_interaction_counts.items():    

    # removes observations that did not have the residue in their msa seqeunce
    real_observations = [x for x in interaction_counts if type(x)==int]

    # skips over those with not enough data
    if len(real_observations) < 12:
        continue

    # calc mean and median count:
    mean = statistics.mean(real_observations)
    median = statistics.median(real_observations)
    stdev = statistics.stdev(real_observations)
    # Coefficient of variation accounts for different size numb interactions. 
    # Other option could be gini inequality. 
    coeff_variation = round(stdev/mean, 3)
    coeff_variantions[msa_res_numb] = coeff_variation
    
print(coeff_variantions)

{5: 0.682, 6: 0.623, 7: 0.621, 8: 0.544, 10: 0.837, 11: 0.715, 12: 0.568, 13: 0.477, 14: 0.724, 15: 0.399, 16: 0.54, 17: 0.367, 18: 0.299, 19: 0.176, 20: 0.468, 21: 0.277, 22: 0.33, 23: 0.514, 24: 1.013, 25: 0.44, 26: 0.42, 27: 0.394, 28: 0.205, 29: 0.228, 30: 0.264, 31: 0.231, 32: 0.286, 33: 0.257, 34: 0.301, 35: 0.353, 36: 0.506, 37: 0.863, 38: 1.354, 39: 0.407, 40: 0.371, 41: 0.398, 42: 0.269, 43: 0.374, 44: 0.158, 45: 0.177, 46: 0.321, 47: 0.491, 48: 0.216, 49: 0.292, 50: 0.207, 51: 0.314, 52: 0.245, 53: 0.323, 54: 0.309, 55: 0.313, 56: 0.257, 57: 0.166, 58: 0.303, 59: 0.266, 60: 0.193, 61: 0.291, 62: 0.246, 63: 0.235, 64: 0.231, 65: 0.202, 66: 0.277, 67: 0.303, 68: 0.257, 69: 0.412, 70: 0.767, 71: 0.42, 73: 0.631, 74: 0.759, 75: 0.499, 76: 0.402, 77: 0.57, 78: 0.244, 79: 0.381, 80: 0.282, 81: 0.595, 82: 0.349, 83: 0.449, 89: 0.975, 90: 2.318, 91: 0.659, 92: 0.272, 93: 0.173, 94: 0.6, 95: 0.176, 96: 0.191, 97: 0.368, 98: 0.155, 99: 0.283, 100: 0.191, 101: 0.725, 102: 0.24, 103: 0.3

#### Step 3 - Compare to alingment data

Read in and process the files...

In [35]:
MSA_SCORES_FILE = r"../../msa_scores/align_and_score.out" 

In [36]:
msa_scores = {}
skip_line = True
wrong_section = True
start_end_line_count = 0 
with open(MSA_SCORES_FILE, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip().split(" ")
        
        # these mark the start and end of the section of interest. 
        if line[0] == "group:":
            wrong_section = False

        if wrong_section:
            continue

        if "------------------------------------" in line[0]:
            start_end_line_count += 1

        if start_end_line_count == 1:
            skip_line = False
        if start_end_line_count == 2:
            skip_line = True
        
        if wrong_section or skip_line:
            continue
        
        try:
            res_numb = int(line[0])
        except ValueError: # happens for first line which is "-", this is okay!
            continue
        
        score = float(line[4])
        msa_scores[res_numb] = score
print(msa_scores)

{1: 0.0, 2: 0.0, 3: 0.28, 4: 0.0, 5: 0.27, 6: 0.0, 7: 0.35, 8: 0.0, 9: 0.5, 10: 0.0, 11: 0.0, 12: 0.55, 13: 0.46, 14: 0.0, 15: 0.0, 16: 0.59, 17: 0.0, 18: 0.61, 19: 0.59, 20: 0.55, 21: 0.5, 22: 0.82, 23: 0.47, 24: 0.54, 25: 0.5, 26: 0.7, 27: 0.7, 28: 0.75, 29: 0.7, 30: 0.96, 31: 0.42, 32: 0.46, 33: 0.46, 34: 0.71, 35: 0.76, 36: 0.45, 37: 0.5, 38: 0.64, 39: 0.76, 40: 0.61, 41: 0.44, 42: 0.67, 43: 0.13, 44: 0.56, 45: 0.56, 46: 0.81, 47: 0.51, 48: 0.58, 49: 0.78, 50: 0.82, 51: 0.9, 52: 0.73, 53: 0.64, 54: 0.28, 55: 0.89, 56: 0.83, 57: 0.7, 58: 0.86, 59: 0.62, 60: 0.64, 61: 0.7, 62: 0.38, 63: 0.65, 64: 0.75, 65: 0.83, 66: 0.87, 67: 0.54, 68: 0.61, 69: 0.51, 70: 0.0, 71: 0.62, 72: 0.58, 73: 0.56, 74: 0.49, 75: 0.57, 76: 0.43, 77: 0.81, 78: 0.5, 79: 0.61, 80: 0.61, 81: 0.76, 82: 0.45, 83: 0.0, 84: 0.72, 85: 0.57, 86: 0.53, 87: 0.56, 88: 0.73, 89: 0.83, 90: 0.0, 91: 0.0, 92: 0.76, 93: 0.0, 94: 0.53, 95: 0.0, 96: 0.7, 97: 0.79, 98: 0.0, 99: 0.0, 100: 0.92, 101: 0.74, 102: 0.74, 103: 0.75, 104:

In [45]:
combined_variance_msa = {}
for msa_res_numb, coeff_variation in coeff_variantions.items():

    res_msa_score = msa_scores[msa_res_numb] 

    combined_variance_msa[msa_res_numb] = [coeff_variation, res_msa_score]
print(combined_variance_msa)

{5: [0.682, 0.27], 6: [0.623, 0.0], 7: [0.621, 0.35], 8: [0.544, 0.0], 10: [0.837, 0.0], 11: [0.715, 0.0], 12: [0.568, 0.55], 13: [0.477, 0.46], 14: [0.724, 0.0], 15: [0.399, 0.0], 16: [0.54, 0.59], 17: [0.367, 0.0], 18: [0.299, 0.61], 19: [0.176, 0.59], 20: [0.468, 0.55], 21: [0.277, 0.5], 22: [0.33, 0.82], 23: [0.514, 0.47], 24: [1.013, 0.54], 25: [0.44, 0.5], 26: [0.42, 0.7], 27: [0.394, 0.7], 28: [0.205, 0.75], 29: [0.228, 0.7], 30: [0.264, 0.96], 31: [0.231, 0.42], 32: [0.286, 0.46], 33: [0.257, 0.46], 34: [0.301, 0.71], 35: [0.353, 0.76], 36: [0.506, 0.45], 37: [0.863, 0.5], 38: [1.354, 0.64], 39: [0.407, 0.76], 40: [0.371, 0.61], 41: [0.398, 0.44], 42: [0.269, 0.67], 43: [0.374, 0.13], 44: [0.158, 0.56], 45: [0.177, 0.56], 46: [0.321, 0.81], 47: [0.491, 0.51], 48: [0.216, 0.58], 49: [0.292, 0.78], 50: [0.207, 0.82], 51: [0.314, 0.9], 52: [0.245, 0.73], 53: [0.323, 0.64], 54: [0.309, 0.28], 55: [0.313, 0.89], 56: [0.257, 0.83], 57: [0.166, 0.7], 58: [0.303, 0.86], 59: [0.266, 0.6

In [46]:
msa_variance_df = pd.DataFrame.from_dict(combined_variance_msa).T
msa_variance_df.columns = ["Coeff of Variation", "MSA value"]
msa_variance_df

,Coeff of Variation,MSA value
5,0.682,0.27
6,0.623,0.00
7,0.621,0.35
8,0.544,0.00
10,0.837,0.00
...,...,...
287,0.664,0.71
288,0.567,0.49
291,0.945,0.56
292,0.938,0.00


In [47]:
msa_variance_df.corr()

,Coeff of Variation,MSA value
Coeff of Variation,1.000000,-0.044621
MSA value,-0.044621,1.000000


In [48]:
import plotly.express as px
fig = px.scatter(msa_variance_df, x="MSA value", y="Coeff of Variation")
fig.show()

## Convert msa residues to tem1 resdiues and make a plot 

In [49]:
print(coeff_variantions)

{5: 0.682, 6: 0.623, 7: 0.621, 8: 0.544, 10: 0.837, 11: 0.715, 12: 0.568, 13: 0.477, 14: 0.724, 15: 0.399, 16: 0.54, 17: 0.367, 18: 0.299, 19: 0.176, 20: 0.468, 21: 0.277, 22: 0.33, 23: 0.514, 24: 1.013, 25: 0.44, 26: 0.42, 27: 0.394, 28: 0.205, 29: 0.228, 30: 0.264, 31: 0.231, 32: 0.286, 33: 0.257, 34: 0.301, 35: 0.353, 36: 0.506, 37: 0.863, 38: 1.354, 39: 0.407, 40: 0.371, 41: 0.398, 42: 0.269, 43: 0.374, 44: 0.158, 45: 0.177, 46: 0.321, 47: 0.491, 48: 0.216, 49: 0.292, 50: 0.207, 51: 0.314, 52: 0.245, 53: 0.323, 54: 0.309, 55: 0.313, 56: 0.257, 57: 0.166, 58: 0.303, 59: 0.266, 60: 0.193, 61: 0.291, 62: 0.246, 63: 0.235, 64: 0.231, 65: 0.202, 66: 0.277, 67: 0.303, 68: 0.257, 69: 0.412, 70: 0.767, 71: 0.42, 73: 0.631, 74: 0.759, 75: 0.499, 76: 0.402, 77: 0.57, 78: 0.244, 79: 0.381, 80: 0.282, 81: 0.595, 82: 0.349, 83: 0.449, 89: 0.975, 90: 2.318, 91: 0.659, 92: 0.272, 93: 0.173, 94: 0.6, 95: 0.176, 96: 0.191, 97: 0.368, 98: 0.155, 99: 0.283, 100: 0.191, 101: 0.725, 102: 0.24, 103: 0.3

In [58]:
msa_sequences["1M40_TEM-1"]

'----HPE---TLVKV--KDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAG-QEQLGRRIHYS-----QNDLVEYSPVTEKHLTDG-MTVRELCSAAITMSDNTAANLLLTTI-GGPKE---LTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGE-LLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGA-GERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW----------*'

In [60]:
tem1_coeff_variations = {}
pdb_res_number = 0 
for msa_index, tem_res in enumerate(msa_sequences["1M40_TEM-1"], start=1):
    
    if tem_res == "-": 
        continue
    else:
        pdb_res_number += 1
    
    try:
        coeff_variation = coeff_variantions[msa_index]
    except KeyError:
        continue
        # can happen if not enough observations of this residue.  

    # convert the msa_index to the pdb_res number. 
    tem1_coeff_variations[pdb_res_number] = coeff_variation
print(tem1_coeff_variations)

{1: 0.682, 2: 0.623, 3: 0.621, 4: 0.715, 5: 0.568, 6: 0.477, 7: 0.724, 8: 0.399, 9: 0.299, 10: 0.176, 11: 0.468, 12: 0.277, 13: 0.33, 14: 0.514, 15: 1.013, 16: 0.44, 17: 0.42, 18: 0.394, 19: 0.205, 20: 0.228, 21: 0.264, 22: 0.231, 23: 0.286, 24: 0.257, 25: 0.301, 26: 0.353, 27: 0.506, 28: 0.863, 29: 1.354, 30: 0.407, 31: 0.371, 32: 0.398, 33: 0.269, 34: 0.374, 35: 0.158, 36: 0.177, 37: 0.321, 38: 0.491, 39: 0.216, 40: 0.292, 41: 0.207, 42: 0.314, 43: 0.245, 44: 0.323, 45: 0.309, 46: 0.313, 47: 0.257, 48: 0.166, 49: 0.303, 50: 0.266, 51: 0.193, 52: 0.291, 53: 0.246, 54: 0.235, 55: 0.231, 56: 0.202, 57: 0.277, 58: 0.303, 59: 0.257, 60: 0.412, 61: 0.767, 62: 0.42, 63: 0.631, 64: 0.759, 65: 0.499, 66: 0.402, 67: 0.57, 68: 0.244, 69: 0.381, 70: 0.282, 71: 0.595, 72: 0.349, 73: 0.449, 74: 0.975, 75: 2.318, 76: 0.659, 77: 0.272, 78: 0.173, 79: 0.6, 80: 0.176, 81: 0.191, 82: 0.368, 83: 0.155, 84: 0.283, 85: 0.191, 86: 0.725, 87: 0.24, 88: 0.319, 89: 1.074, 90: 0.663, 91: 0.202, 92: 0.113, 93: 

In [62]:
from tools_proj.pymol_projections import project_pymol_per_res_scores
project_pymol_per_res_scores(
    per_res_scores=tem1_coeff_variations,
    out_file= r"../../comparitive_data/comparison_results/tem1_coeff_variation.py"
)

The file: ../../comparitive_data/comparison_results/tem1_coeff_variation.py was written to disk.
